In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("C:\\Users\\Administrator\\Downloads\\mw_pw_profiles.txt",low_memory=False)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df.head()

,player_id,match_id,gender,balls_per_over,start_date,series_name,match_type,name_x,runs_scored,player_out,balls_faced,fours_scored,sixes_scored,catches_taken,run_out_direct,run_out_throw,stumpings_done,out_kind,dot_balls_as_batsman,order_seen,balls_bowled,runs_conceded,wickets_taken,bowled_done,lbw_done,maidens,dot_balls_as_bowler,player_team,opposition_team,name_y,unique_name,key_bcci,key_bcci_2,key_bigbash,key_cricbuzz,key_cricheroes,key_crichq,key_cricinfo,key_cricinfo_2,key_cricinfo_3,key_cricingif,key_cricketarchive,key_cricketarchive_2,key_cricketworld,key_nvplay,key_nvplay_2,key_opta,key_opta_2,key_pulse,key_pulse_2,full_name,batting_style,bowling_style,playing_role,teams,fantasy_score_batting,fantasy_score_bowling,fantasy_score_total
0,00015688,1158348,female,6,2018-08-21,Botswana Cricket Association Women's T20I Series,T20,M King,10,1,30,1,0,1,0,0,0,run out,24,1,0,0,0,0,0,0,0,Sierra Leone,Namibia,M King,M King,NaN,NaN,NaN,NaN,NaN,NaN,1158425.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],5,8,17
1,00015688,1182644,female,6,2019-05-05,ICC Women's T20 World Cup Africa Region Qualifier,T20,M King,8,1,18,1,0,0,0,0,0,0,14,0,0,0,0,0,0,0,0,Sierra Leone,Uganda,M King,M King,NaN,NaN,NaN,NaN,NaN,NaN,1158425.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],3,0,7
2,00015688,1275107,female,6,2021-09-09,ICC Women's T20 World Cup Africa Region Qualifier,T20,M King,1,0,5,0,0,0,0,0,0,0,4,10,0,0,0,0,0,0,0,Sierra Leone,Nigeria,M King,M King,NaN,NaN,NaN,NaN,NaN,NaN,1158425.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],1,0,5
3,00015688,1275113,female,6,2021-09-15,ICC Women's T20 World Cup Africa Region Qualifier,T20,M King,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Sierra Leone,Cameroon,M King,M King,NaN,NaN,NaN,NaN,NaN,NaN,1158425.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],0,0,4
4,00015688,1275125,female,6,2021-09-14,ICC Women's T20 World Cup Africa Region Qualifier,T20,M King,1,1,15,0,0,0,0,0,0,caught and bowled,14,4,0,0,0,0,0,0,0,Sierra Leone,Uganda,M King,M King,NaN,NaN,NaN,NaN,NaN,NaN,1158425.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],-5,0,-1


In [4]:
df_t20 = df[df['match_type'].isin(['T20', 'IT20'])]

In [5]:
df_t20['start_date'] = pd.to_datetime(df_t20['start_date'], errors='coerce')
df_t20 = df_t20.sort_values(by=['player_id', 'start_date'])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_41972\2796886576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t20['start_date'] = pd.to_datetime(df_t20['start_date'], errors='coerce')


In [6]:
df_t20['strike_rate'] = np.where(df_t20['balls_faced'] != 0, (df_t20['runs_scored'] / df_t20['balls_faced']) * 100, 0)
df_t20['total_boundaries']=df_t20['fours_scored']+df_t20['sixes_scored']
df_t20['run_out_direct']=df_t20['run_out_direct']+df_t20['run_out_throw']
df_t20['fielding']=df_t20['catches_taken']+df_t20['run_out_direct']+df_t20['stumpings_done']
df_t20['economy'] = np.where(df_t20['balls_bowled'] > 0,(df_t20['runs_conceded'] * df_t20['balls_per_over']) / df_t20['balls_bowled'],0)
df_t20['is_fifty'] = df_t20['runs_scored'].apply(lambda x: 1 if 50 <= x < 100 else 0)
df_t20['is_century'] = df_t20['runs_scored'].apply(lambda x: 1 if x >= 100 else 0)

In [9]:
df_t20['boundary_ratio'] = np.where(df_t20['balls_faced'] != 0, (df_t20['total_boundaries']/ df_t20['balls_faced']), 0)

df_t20['batting_aggression'] = np.where(df_t20['balls_faced'] != 0, ((df_t20['fours_scored'] + 2 * df_t20['sixes_scored']) / df_t20['balls_faced']), 0)

df_t20['bowling_strike_rate'] = np.where(df_t20['wickets_taken'] > 0,(df_t20['balls_bowled'] / df_t20['wickets_taken']),0)
                                         
df_t20['dotball_percent_bowler'] = np.where(df_t20['balls_bowled'] > 0,(df_t20['dot_balls_as_bowler'] / df_t20['balls_bowled']),0)

df_t20['dotball_percent_batsman'] =  np.where(df_t20['balls_faced'] != 0, (df_t20['dot_balls_as_batsman'] / df_t20['balls_faced']), 0)

df_t20['impact_score'] = (
    df_t20['runs_scored'] * 0.5 +
    df_t20['wickets_taken'] * 12 +
    df_t20['fielding']
)


In [10]:
df_t20['order_seen'].value_counts()

order_seen
0     164007
1      11771
2      11769
3      11741
4      11636
5      11309
6      10515
7       9143
8       7268
9       5230
10      3465
11      1848
12         3
Name: count, dtype: int64

In [11]:
df_t20 = df_t20[df_t20['order_seen'] != 12]

In [12]:
order_seen_filtered = df_t20['order_seen'].where(df_t20['order_seen'] != 0)

df_t20['rolling_order'] = (
    df_t20.groupby('player_id')[order_seen_filtered.name]
      .transform(lambda x: x.dropna().rolling(window=3, min_periods=1).mean())
)

df_t20['rolling_bowled'] = (
    df_t20.groupby('player_id')['balls_bowled']
      .transform(lambda x: x.rolling(window=3, min_periods=1).mean())
)

def classify_role(row):
    if pd.isna(row['rolling_order']) and row['rolling_bowled'] > 6:
        return 'bowler'
    elif pd.isna(row['rolling_bowled']) and row['rolling_order'] > 0:
        return 'batsman'
    elif row['rolling_order'] <= 5 and row['rolling_bowled'] < 6:
        return 'batsman'
    elif row['rolling_order'] >= 6 and row['rolling_bowled'] >= 6:
        return 'all-rounder'
    elif row['rolling_bowled'] >= 12:
        return 'bowler'
    elif row['rolling_order'] <= 6:
        return 'batsman'
    else:
        return 'all-rounder'

df_t20['player_role'] = df_t20.apply(classify_role, axis=1)

role_dummies = pd.get_dummies(df_t20['player_role'], prefix='role',dtype=int)
df_t20 = pd.concat([df_t20, role_dummies], axis=1)

In [13]:
df_t20['player_role'].value_counts()

player_role
batsman        133807
bowler         117696
all-rounder      8199
Name: count, dtype: int64

In [52]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df_t20['player_team_encoded'] = le.fit_transform(df_t20['player_team'].astype(str))
df_t20['opposition_team_encoded'] = le.fit_transform(df_t20['opposition_team'].astype(str))

In [14]:
df_t20=df_t20.drop(['order_seen','stumpings_done','balls_per_over','key_cricinfo','gender','run_out_direct','catches_taken','run_out_throw','lbw_done','bowled_done','teams','out_kind','name_y','unique_name','run_out_throw','key_bcci','key_bcci_2','key_bigbash','key_cricbuzz','key_cricheroes','key_crichq','key_cricinfo_2','key_cricinfo_3','key_cricingif','key_cricketarchive','key_cricketarchive_2','key_cricketworld','key_nvplay','key_nvplay_2','key_opta','key_opta_2','key_pulse','key_pulse_2','full_name','batting_style','bowling_style','playing_role'],axis=1,errors='ignore')

In [53]:
df_t20.head()

,player_id,match_id,start_date,series_name,match_type,name_x,runs_scored,player_out,balls_faced,fours_scored,sixes_scored,dot_balls_as_batsman,balls_bowled,runs_conceded,wickets_taken,maidens,dot_balls_as_bowler,player_team,opposition_team,fantasy_score_batting,fantasy_score_bowling,fantasy_score_total,strike_rate,total_boundaries,fielding,economy,is_fifty,is_century,boundary_ratio,batting_aggression,bowling_strike_rate,dotball_percent_bowler,dotball_percent_batsman,impact_score,rolling_order,rolling_bowled,player_role,role_all-rounder,role_batsman,role_bowler,player_team_encoded,opposition_team_encoded
0,00015688,1158348,2018-08-21,Botswana Cricket Association Women's T20I Series,T20,M King,10,1,30,1,0,24,0,0,0,0,0,Sierra Leone,Namibia,5,8,17,33.333333,1,1,0.0,0,0,0.033333,0.033333,0.0,0.0,0.800000,6.0,1.000000,0.0,batsman,0,1,0,268,207
1,00015688,1182644,2019-05-05,ICC Women's T20 World Cup Africa Region Qualifier,T20,M King,8,1,18,1,0,14,0,0,0,0,0,Sierra Leone,Uganda,3,0,7,44.444444,1,0,0.0,0,0,0.055556,0.055556,0.0,0.0,0.777778,4.0,0.500000,0.0,batsman,0,1,0,268,324
2,00015688,1275107,2021-09-09,ICC Women's T20 World Cup Africa Region Qualifier,T20,M King,1,0,5,0,0,4,0,0,0,0,0,Sierra Leone,Nigeria,1,0,5,20.000000,0,0,0.0,0,0,0.000000,0.000000,0.0,0.0,0.800000,0.5,3.666667,0.0,batsman,0,1,0,268,212
4,00015688,1275125,2021-09-14,ICC Women's T20 World Cup Africa Region Qualifier,T20,M King,1,1,15,0,0,14,0,0,0,0,0,Sierra Leone,Uganda,-5,0,-1,6.666667,0,0,0.0,0,0,0.000000,0.000000,0.0,0.0,0.933333,0.5,4.666667,0.0,batsman,0,1,0,268,324
3,00015688,1275113,2021-09-15,ICC Women's T20 World Cup Africa Region Qualifier,T20,M King,0,0,0,0,0,0,0,0,0,0,0,Sierra Leone,Cameroon,0,0,4,0.000000,0,0,0.0,0,0,0.000000,0.000000,0.0,0.0,0.000000,0.0,4.666667,0.0,batsman,0,1,0,268,37


In [54]:
pd.set_option('display.max_columns', None)
df_t20[df_t20['player_id']=='ba607b88'].head()

,player_id,match_id,start_date,series_name,match_type,name_x,runs_scored,player_out,balls_faced,fours_scored,sixes_scored,dot_balls_as_batsman,balls_bowled,runs_conceded,wickets_taken,maidens,dot_balls_as_bowler,player_team,opposition_team,fantasy_score_batting,fantasy_score_bowling,fantasy_score_total,strike_rate,total_boundaries,fielding,economy,is_fifty,is_century,boundary_ratio,batting_aggression,bowling_strike_rate,dotball_percent_bowler,dotball_percent_batsman,impact_score,rolling_order,rolling_bowled,player_role,role_all-rounder,role_batsman,role_bowler,player_team_encoded,opposition_team_encoded
299534,ba607b88,335982,2008-04-18,Indian Premier League,T20,V Kohli,1,1,5,0,0,4,0,0,0,0,0,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,5,20.000000,0,0,0.0,0,0,0.000000,0.000000,0.0,0.0,0.800000,0.5,0.000000,0.0,batsman,0,1,0,253,162
299535,ba607b88,335985,2008-04-20,Indian Premier League,T20,V Kohli,23,1,15,3,1,6,0,0,0,0,0,Royal Challengers Bangalore,Mumbai Indians,32,16,52,153.333333,4,2,0.0,0,0,0.266667,0.333333,0.0,0.0,0.400000,13.5,0.000000,0.0,batsman,0,1,0,253,203
299536,ba607b88,335992,2008-04-26,Indian Premier League,T20,V Kohli,13,1,16,0,1,9,0,0,0,0,0,Royal Challengers Bangalore,Rajasthan Royals,15,0,19,81.250000,1,0,0.0,0,0,0.062500,0.125000,0.0,0.0,0.562500,6.5,1.666667,0.0,batsman,0,1,0,253,245
299537,ba607b88,335996,2008-04-28,Indian Premier League,T20,V Kohli,12,1,7,2,0,2,0,0,0,0,0,Royal Challengers Bangalore,Chennai Super Kings,14,0,18,171.428571,2,0,0.0,0,0,0.285714,0.285714,0.0,0.0,0.285714,6.0,1.666667,0.0,batsman,0,1,0,253,47
299538,ba607b88,335998,2008-04-30,Indian Premier League,T20,V Kohli,1,0,1,0,0,0,6,11,0,0,0,Royal Challengers Bangalore,Delhi Daredevils,1,0,5,100.000000,0,0,11.0,0,0,0.000000,0.000000,0.0,0.0,0.000000,0.5,1.666667,2.0,batsman,0,1,0,253,71


In [55]:
numeric_df = df_t20.select_dtypes(include='number')
corr_matrix = numeric_df.corr()
corr_matrix

,runs_scored,player_out,balls_faced,fours_scored,sixes_scored,dot_balls_as_batsman,balls_bowled,runs_conceded,wickets_taken,maidens,dot_balls_as_bowler,fantasy_score_batting,fantasy_score_bowling,fantasy_score_total,strike_rate,total_boundaries,fielding,economy,is_fifty,is_century,boundary_ratio,batting_aggression,bowling_strike_rate,dotball_percent_bowler,dotball_percent_batsman,impact_score,rolling_order,rolling_bowled,role_all-rounder,role_batsman,role_bowler,player_team_encoded,opposition_team_encoded
runs_scored,1.000000,0.259947,0.921181,0.878265,0.747337,0.712382,-0.282489,-0.257791,-0.172237,-0.049189,-0.260842,0.991089,-0.153306,0.596560,0.634128,0.954354,0.081044,-0.231590,0.680923,0.293870,0.548507,0.545602,-0.179809,-0.264905,0.052265,0.571954,0.002137,-0.311029,-0.051822,0.300742,-0.283726,-0.004355,-0.004648
player_out,0.259947,1.000000,0.364414,0.237886,0.137165,0.441436,-0.336974,-0.303917,-0.228200,-0.071286,-0.311037,0.227426,-0.214508,-0.006971,0.323499,0.232642,0.047573,-0.260673,0.061341,-0.009769,0.246486,0.221741,-0.222355,-0.309566,0.678180,-0.002914,0.112310,-0.364583,0.003728,0.343922,-0.346588,-0.006974,0.003602
balls_faced,0.921181,0.364414,1.000000,0.794441,0.562630,0.902629,-0.320885,-0.304073,-0.202655,-0.051543,-0.286359,0.879756,-0.180889,0.493290,0.519663,0.818350,0.076337,-0.272322,0.588666,0.202249,0.400399,0.380985,-0.206980,-0.288791,0.205786,0.490264,0.011529,-0.351401,-0.048810,0.337197,-0.321382,-0.004016,-0.000833
fours_scored,0.878265,0.237886,0.794441,1.000000,0.473602,0.637313,-0.262211,-0.240946,-0.157941,-0.042956,-0.239349,0.876015,-0.140264,0.523611,0.564111,0.937151,0.073900,-0.220994,0.604107,0.252608,0.599135,0.512923,-0.165913,-0.245104,0.064326,0.496908,-0.043214,-0.288680,-0.056938,0.279205,-0.260307,-0.001412,-0.001338
sixes_scored,0.747337,0.137165,0.562630,0.473602,1.000000,0.403482,-0.161151,-0.134313,-0.094832,-0.034195,-0.157241,0.778324,-0.086095,0.496099,0.534221,0.751148,0.052317,-0.116563,0.528946,0.319547,0.484023,0.599122,-0.102042,-0.159300,0.005444,0.455268,0.014416,-0.178426,-0.030766,0.174932,-0.164815,-0.005132,-0.009757
dot_balls_as_batsman,0.712382,0.441436,0.902629,0.637313,0.403482,1.000000,-0.315540,-0.307598,-0.203870,-0.045377,-0.270634,0.659704,-0.183000,0.331502,0.357849,0.637529,0.058395,-0.275512,0.412227,0.121821,0.291333,0.267613,-0.205676,-0.271303,0.400564,0.339774,-0.001778,-0.343739,-0.041473,0.328231,-0.314958,-0.003062,0.001261
balls_bowled,-0.282489,-0.336974,-0.320885,-0.262211,-0.161151,-0.315540,1.000000,0.886398,0.619917,0.146972,0.901003,-0.273400,0.588566,0.273470,-0.240507,-0.260377,-0.116892,0.689058,-0.136299,-0.036466,-0.199511,-0.178059,0.666337,0.814966,-0.262014,0.309931,-0.005894,0.919943,0.060485,-0.841661,0.823735,-0.001066,-0.004834
runs_conceded,-0.257791,-0.303917,-0.304073,-0.240946,-0.134313,-0.307598,0.886398,1.000000,0.460075,0.013388,0.664108,-0.247229,0.387292,0.131637,-0.200759,-0.233805,-0.114209,0.853485,-0.124708,-0.032354,-0.164870,-0.142469,0.558150,0.605348,-0.241348,0.194307,-0.011920,0.834223,0.052829,-0.767075,0.751544,-0.006431,-0.005937
wickets_taken,-0.172237,-0.228200,-0.202655,-0.157941,-0.094832,-0.203870,0.619917,0.460075,1.000000,0.207710,0.658423,-0.164563,0.965232,0.654294,-0.155175,-0.155950,-0.071882,0.308804,-0.079179,-0.020391,-0.122561,-0.108108,0.575784,0.588177,-0.190419,0.707813,-0.014301,0.557398,0.028108,-0.506126,0.498249,0.000467,-0.004719
maidens,-0.049189,-0.071286,-0.051543,-0.042956,-0.034195,-0.045377,0.146972,0.013388,0.207710,1.000000,0.275383,-0.047978,0.293243,0.199770,-0.056877,-0.045743,-0.012212,-0.009353,-0.020779,-0.004860,-0.043932,-0.042318,0.108807,0.274727,-0.052588,0.137624,-0.001876,0.140379,0.011212,-0.130474,0.127051,0.006421,-0.002201


In [56]:
df_enriched=df_t20[['player_id','start_date','match_id','player_role','rolling_order','rolling_bowled','fantasy_score_total','player_team_encoded','opposition_team_encoded']]
df_enriched = df_enriched.sort_values(by=['player_id', 'start_date'])

In [57]:
rolling_features = [
    'runs_scored', 'balls_faced', 'total_boundaries','balls_bowled','maidens','fantasy_score_batting','fantasy_score_bowling','fielding','batting_aggression','bowling_strike_rate','dotball_percent_bowler','dotball_percent_batsman','impact_score',
    'dot_balls_as_batsman', 'wickets_taken', 'dot_balls_as_bowler',
    'runs_conceded', 'fantasy_score_total', 'strike_rate', 'economy','boundary_ratio',
]

window_size = 3

for col in rolling_features:
    df_enriched[f'rolling3_{col}'] = (
        df_t20
        .sort_values(['player_id', 'start_date'])
        .groupby('player_id')[col]
        .transform(lambda x: x.shift(1).rolling(window=window_size, min_periods=1).mean())
    )

In [58]:
df_enriched = df_enriched.groupby('player_id', group_keys=False).apply(lambda g: g.iloc[1:]).reset_index(drop=True)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_41972\1967713865.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_enriched = df_enriched.groupby('player_id', group_keys=False).apply(lambda g: g.iloc[1:]).reset_index(drop=True)


In [59]:
df_enriched.head()

,player_id,start_date,match_id,player_role,rolling_order,rolling_bowled,fantasy_score_total,player_team_encoded,opposition_team_encoded,rolling3_runs_scored,rolling3_balls_faced,rolling3_total_boundaries,rolling3_balls_bowled,rolling3_maidens,rolling3_fantasy_score_batting,rolling3_fantasy_score_bowling,rolling3_fielding,rolling3_batting_aggression,rolling3_bowling_strike_rate,rolling3_dotball_percent_bowler,rolling3_dotball_percent_batsman,rolling3_impact_score,rolling3_dot_balls_as_batsman,rolling3_wickets_taken,rolling3_dot_balls_as_bowler,rolling3_runs_conceded,rolling3_fantasy_score_total,rolling3_strike_rate,rolling3_economy,rolling3_boundary_ratio
0,00015688,2019-05-05,1182644,batsman,0.500000,0.0,7,268,324,10.000000,30.000000,1.000000,0.0,0.0,5.000000,8.000000,1.000000,0.033333,0.0,0.0,0.800000,6.000000,24.000000,0.0,0.0,0.0,17.000000,33.333333,0.0,0.033333
1,00015688,2021-09-09,1275107,batsman,3.666667,0.0,5,268,212,9.000000,24.000000,1.000000,0.0,0.0,4.000000,4.000000,0.500000,0.044444,0.0,0.0,0.788889,5.000000,19.000000,0.0,0.0,0.0,12.000000,38.888889,0.0,0.044444
2,00015688,2021-09-14,1275125,batsman,4.666667,0.0,-1,268,324,6.333333,17.666667,0.666667,0.0,0.0,3.000000,2.666667,0.333333,0.029630,0.0,0.0,0.792593,3.500000,14.000000,0.0,0.0,0.0,9.666667,32.592593,0.0,0.029630
3,00015688,2021-09-15,1275113,batsman,4.666667,0.0,4,268,37,3.333333,12.666667,0.333333,0.0,0.0,-0.333333,0.000000,0.000000,0.018519,0.0,0.0,0.837037,1.666667,10.666667,0.0,0.0,0.0,3.666667,23.703704,0.0,0.018519
4,00029c30,2021-10-16,1283024,batsman,4.500000,6.0,4,185,324,0.000000,0.000000,0.000000,12.0,0.0,0.000000,33.000000,0.000000,0.000000,12.0,0.5,0.000000,12.000000,0.000000,1.0,6.0,16.0,33.000000,0.000000,8.0,0.000000


In [60]:
numeric_df = df_enriched.select_dtypes(include='number')
corr = numeric_df.corr()
corr

,rolling_order,rolling_bowled,fantasy_score_total,player_team_encoded,opposition_team_encoded,rolling3_runs_scored,rolling3_balls_faced,rolling3_total_boundaries,rolling3_balls_bowled,rolling3_maidens,rolling3_fantasy_score_batting,rolling3_fantasy_score_bowling,rolling3_fielding,rolling3_batting_aggression,rolling3_bowling_strike_rate,rolling3_dotball_percent_bowler,rolling3_dotball_percent_batsman,rolling3_impact_score,rolling3_dot_balls_as_batsman,rolling3_wickets_taken,rolling3_dot_balls_as_bowler,rolling3_runs_conceded,rolling3_fantasy_score_total,rolling3_strike_rate,rolling3_economy,rolling3_boundary_ratio
rolling_order,1.000000,0.001113,-0.014896,-0.015966,0.007602,-0.020559,-0.015838,-0.054107,0.010435,0.001445,-0.027700,-0.001486,-0.006634,0.068585,-0.004437,0.038828,0.133220,-0.018537,-0.034988,-0.002266,0.012002,0.007503,-0.023593,0.162637,0.045063,0.050416
rolling_bowled,0.001113,1.000000,0.191607,-0.001211,-0.003006,-0.437486,-0.476164,-0.406228,0.964201,0.218270,-0.428238,0.704167,-0.212880,-0.295035,0.767706,0.869896,-0.390559,0.331570,-0.454471,0.735538,0.902864,0.899398,0.280215,-0.380170,0.798232,-0.327023
fantasy_score_total,-0.014896,0.191607,1.000000,-0.002677,-0.007841,0.093703,0.070497,0.099817,0.135604,0.044767,0.095165,0.125961,0.013817,0.086569,0.111373,0.125359,-0.002310,0.185822,0.043717,0.120079,0.131574,0.120338,0.187386,0.083214,0.102950,0.086617
player_team_encoded,-0.015966,-0.001211,-0.002677,1.000000,-0.039466,-0.009283,-0.008023,-0.007785,-0.000597,0.011535,-0.009190,0.003113,-0.001955,-0.006983,0.003660,0.007696,-0.009894,-0.006888,-0.005298,0.000510,0.002945,-0.006950,-0.005408,-0.007675,-0.004255,-0.005750
opposition_team_encoded,0.007602,-0.003006,-0.007841,-0.039466,1.000000,-0.002829,-0.002495,-0.002983,-0.001476,0.005534,-0.003021,0.000750,-0.002684,-0.002374,0.000127,0.002856,-0.003357,-0.002296,-0.003525,0.000061,0.000703,-0.004220,-0.002522,0.000122,-0.003781,-0.000198
rolling3_runs_scored,-0.020559,-0.437486,0.093703,-0.009283,-0.002829,1.000000,0.931384,0.962199,-0.434792,-0.106091,0.992193,-0.300738,0.158942,0.633734,-0.340366,-0.416310,0.259542,0.484051,0.754693,-0.329054,-0.414396,-0.413857,0.523679,0.720939,-0.384519,0.652873
rolling3_balls_faced,-0.015838,-0.476164,0.070497,-0.008023,-0.002495,0.931384,1.000000,0.845020,-0.473975,-0.102243,0.894053,-0.334520,0.152040,0.478834,-0.375051,-0.437749,0.411731,0.394765,0.920579,-0.366919,-0.438187,-0.466149,0.415158,0.616159,-0.432783,0.516434
rolling3_total_boundaries,-0.054107,-0.406228,0.099817,-0.007785,-0.002983,0.962199,0.845020,1.000000,-0.403681,-0.101758,0.971316,-0.277809,0.147372,0.698049,-0.314744,-0.390226,0.244965,0.478618,0.689793,-0.302486,-0.386516,-0.379374,0.527569,0.714309,-0.354920,0.727248
rolling3_balls_bowled,0.010435,0.964201,0.135604,-0.000597,-0.001476,-0.434792,-0.473975,-0.403681,1.000000,0.221396,-0.425520,0.728878,-0.209767,-0.291902,0.799116,0.889810,-0.390382,0.356959,-0.453126,0.760366,0.935281,0.932931,0.304162,-0.376682,0.816227,-0.323842
rolling3_maidens,0.001445,0.218270,0.044767,0.011535,0.005534,-0.106091,-0.102243,-0.101758,0.221396,1.000000,-0.106106,0.344478,-0.043021,-0.093678,0.184554,0.338523,-0.089529,0.161098,-0.084385,0.264544,0.340309,0.096481,0.217977,-0.115260,0.067026,-0.097391


In [34]:
!pip install pyswarms

In [61]:
X = df_enriched.drop(['player_id','start_date','match_id','player_role','fantasy_score_total'],axis=1)
y = df_enriched['fantasy_score_total']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from pyswarms.single.global_best import GlobalBestPSO
from pyswarms.utils.plotters import plot_cost_history


bounds = (
    [0.01, 3, 100, 0.5, 0.5],    
    [0.3, 8, 1000, 1.0, 1.0]     

def objective_function(params):
    scores = []
    for p in params:
        model = XGBRegressor(
            learning_rate=p[0],
            max_depth=int(p[1]),
            n_estimators=int(p[2]),
            subsample=p[3],
            colsample_bytree=p[4],
            reg_alpha=0.5,
            reg_lambda=1.0,
            objective='reg:squarederror',
            random_state=42,
            n_jobs=-1,
            verbosity=0
        )
        score = cross_val_score(model, X, y, cv=3, scoring='neg_root_mean_squared_error').mean()
        scores.append(-score) 
    return np.array(scores)

optimizer = GlobalBestPSO(
    n_particles=15,
    dimensions=5,
    options={'c1': 0.5, 'c2': 0.3, 'w': 0.9},
    bounds=bounds
)

best_cost, best_params = optimizer.optimize(objective_function, iters=30)

print("\nBest hyperparameters found:")
print(f"Learning Rate: {best_params[0]:.4f}")
print(f"Max Depth: {int(best_params[1])}")
print(f"N Estimators: {int(best_params[2])}")
print(f"Subsample: {best_params[3]:.2f}")
print(f"Colsample_bytree: {best_params[4]:.2f}")


2025-06-23 10:38:48,665 - pyswarms.single.global_best - INFO - Optimize for 30 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|███████████████████████████████████████████████████████████████|30/30, best_cost=31.2
2025-06-23 20:12:44,347 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 31.177070811106148, best pos: [1.60244248e-02 5.73727717e+00 7.71526545e+02 7.99127861e-01
 7.03778136e-01]



Best hyperparameters found:
Learning Rate: 0.0160
Max Depth: 5
N Estimators: 771
Subsample: 0.80
Colsample_bytree: 0.70


In [42]:
from sklearn.feature_selection import RFE

model_pso = XGBRegressor(
    learning_rate=best_params[0],
    max_depth=int(best_params[1]),
    n_estimators=int(best_params[2]),
    subsample=best_params[3],
    colsample_bytree=best_params[4],
    reg_alpha=0.5,
    reg_lambda=1.0,
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1,
    verbosity=0
)

rfe_selector = RFE(estimator=model_pso, n_features_to_select=15, step=1)
rfe_selector = rfe_selector.fit(X_train, y_train)

selected_features = X.columns[rfe_selector.support_]
print("\nSelected Features by RFE:")
print(selected_features.tolist())

X_train_rfe = rfe_selector.transform(X_train)
X_test_rfe = rfe_selector.transform(X_test)

model_final = model_pso.fit(X_train_rfe, y_train)
r2 = model_final.score(X_test_rfe, y_test)
print(f"\nFinal R² score after RFE: {r2:.4f}")



Selected Features by RFE:
['rolling_order', 'rolling_bowled', 'rolling3_runs_scored', 'rolling3_balls_faced', 'rolling3_total_boundaries', 'rolling3_balls_bowled', 'rolling3_fantasy_score_batting', 'rolling3_fielding', 'rolling3_batting_aggression', 'rolling3_impact_score', 'rolling3_dot_balls_as_batsman', 'rolling3_dot_balls_as_bowler', 'rolling3_runs_conceded', 'rolling3_fantasy_score_total', 'rolling3_strike_rate']

Final R² score after RFE: 0.1352


In [62]:
model = XGBRegressor(
    learning_rate=best_params[0],
    max_depth=int(best_params[1]),
    n_estimators=int(best_params[2]),
    subsample=best_params[3],
    colsample_bytree=best_params[4],
    reg_alpha=0.5,
    reg_lambda=1.0,
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1,
    verbosity=0
)

print("Training XGBoost...")
model.fit(X_train, y_train)
importances = model.feature_importances_
for feat, imp in sorted(zip(X_train.columns, importances), key=lambda x: -x[1]):
    print(f"{feat}: {imp:.4f}")


Training XGBoost...
rolling_bowled: 0.1496
rolling3_total_boundaries: 0.1027
rolling3_fantasy_score_total: 0.0976
rolling3_balls_bowled: 0.0899
rolling3_fantasy_score_batting: 0.0611
rolling3_runs_scored: 0.0547
rolling3_impact_score: 0.0525
rolling3_batting_aggression: 0.0499
rolling3_runs_conceded: 0.0365
rolling3_dot_balls_as_bowler: 0.0363
rolling3_balls_faced: 0.0350
rolling_order: 0.0340
rolling3_strike_rate: 0.0211
rolling3_dot_balls_as_batsman: 0.0193
rolling3_boundary_ratio: 0.0183
rolling3_dotball_percent_bowler: 0.0178
rolling3_dotball_percent_batsman: 0.0177
rolling3_economy: 0.0166
rolling3_fielding: 0.0165
rolling3_fantasy_score_bowling: 0.0133
rolling3_bowling_strike_rate: 0.0127
rolling3_maidens: 0.0123
opposition_team_encoded: 0.0121
player_team_encoded: 0.0114
rolling3_wickets_taken: 0.0110


In [63]:
y_pred = model.predict(X_test)
from sklearn.metrics import mean_squared_error, r2_score
print(r2_score(y_test, y_pred))

0.13679605722427368


In [64]:
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(y_test, y_pred))

23.7293693491657


In [65]:
y_pred = model.predict(X_train)
from sklearn.metrics import mean_squared_error, r2_score
print(r2_score(y_train, y_pred))

0.15665388107299805


In [33]:
!pip install xgboost

In [34]:
!pip install shap

In [77]:
from sklearn.metrics import mean_absolute_error
import numpy as np

match_counts = df_enriched['match_id'].value_counts()
valid_matches = match_counts[match_counts >= 22].index

m_list = []

for match_id in valid_matches:
    match_df = df_enriched[df_enriched['match_id'] == match_id].copy()
    
    X_match = match_df.drop(columns=['player_id', 'start_date', 'match_id', 'fantasy_score_total', 'player_role'], errors='ignore')
    
    match_df['predicted_score'] = model.predict(X_match)

    top11_actual_ids = match_df.sort_values('fantasy_score_total', ascending=False).head(11)['player_id'].tolist()
    top11_predicted_ids = match_df.sort_values('predicted_score', ascending=False).head(11)['player_id'].tolist()

    correct_matches = len(set(top11_actual_ids).intersection(set(top11_predicted_ids)));
    m_list.append(correct_matches)

In [78]:
mean = np.mean(m_list)
print("avg number of correct predictions in top11:",round(mean,0))
print("avg % mismatches across all matches:", (1-mean/11)*100)

avg number of correct predictions in top11: 7.0
avg % mismatches across all matches: 37.183947500087264
